In [108]:
from abc import abstractmethod, ABC
from json import load
from numbers import Real
from pathlib import Path
from typing import Dict, Iterable, Iterator, Tuple, Union, Any, List, Callable
from enum import Enum
from collections.abc import MutableSequence
import pandas as pd


class Type(Enum):
    Float = 0
    String = 1
    Integer = 'integer'


def to_float(obj) -> float:
    """
    cast object to float with support of None objects (None is cast to None)
    """
    return float(obj) if obj is not None else None


def to_str(obj) -> str:
    """
    cast object to float with support of None objects (None is cast to None)
    """
    return str(obj) if obj is not None else None


def common(iterator): # from ChatGPT
    try:
        # Nejprve zkusíme získat první prvek iterátoru
        iterator = iter(iterator)
        first_value = next(iterator)
    except StopIteration:
        # Vyvolá výjimku, pokud je iterátor prázdný
        raise ValueError("Iterator is empty")

    # Kontrola, zda jsou všechny další prvky stejné jako první prvek
    for value in iterator:
        if value != first_value:
            raise ValueError("Not all values are the same")

    # Vrací hodnotu, pokud všechny prvky jsou stejné
    return first_value


class Column(MutableSequence):  # implement MutableSequence (some method are mixed from abc)
    def __init__(self, data: Iterable, dtype: Type):
        self.dtype = dtype
        self._cast = to_float if self.dtype == Type.Float else to_str # cast function
        self._data = [self._cast(value) for value in data]

    def __len__(self) -> int:
        return len(self._data)

    def __getitem__(self, item: Union[int, slice]) -> Union[float, str]:
        return self._data[item]

    def __setitem__(self, key: Union[int, slice], value: Any) -> None:
        self._data[key] = self._cast(value)

    def append(self, item: Any) -> None:
        self._data.append(self._cast(item))

    def insert(self, index: int, value: Any) -> None:
        self._data.insert(index, self._cast(value))

    def __delitem__(self, index: Union[int, slice]) -> None:
        del self._data[index]

    def permute(self, indices: List[int]):
        # vytvořit nový sloupec
        # projít přes všechny zadané indexy
        # pro každý index vyberete odpovídající hodnotu sloupce self._data
        # hodnotu přidejte do nově vytvořeného sloupce

        column=Column([],self.dtype) # vytvořím nový prázdný sloupec stejného datového typu
        for index in indices:
          column.append(self._data[index])
        return column

    def copy(self) -> 'Column':
        # FIXME: value is casted to the same type (minor optimisation problem)
        return Column(self._data, self.dtype)

    def get_formatted_item(self, index:int, *, width: int):
        assert width > 0
        if self._data[index] is None:
            return "n/a".rjust(width)
        return format(self._data[index],
                      f"{width}s" if self.dtype == Type.String else f"-{width}.2g")


class DataFrame:
    def __init__(self, columns: Dict[str, Column]):
        """
        :param columns: columns of dataframe (key: name of dataframe),
                        lengths of all columns has to be the same
        """
        assert len(columns) > 0, "Dataframe without columns is not supported"
        self._size = common(len(column) for column in columns.values())
        # deep copy od dict `columns`
        self._columns = {name: column.copy() for name, column in columns.items()}

    def __getitem__(self, index: int) -> Tuple[Union[str,float]]:
        pass

    def __iter__(self) -> Iterator[Tuple[Union[str, float]]]:
        """
        :return: iterator over lines of dataframe
        """
        for i in range(len(self)):
            yield tuple(c[i] for c in self._columns.values())

    def __len__(self) -> int:
        return self._size

    @property
    def columns(self) -> Iterable[str]:
        return self._columns.keys()

    def __repr__(self) -> str:
        lines = []
        lines.append(" ".join(f"{name:12s}" for name in self.columns))
        for i in range(len(self)):
            lines.append(" ".join(self._columns[cname].get_formatted_item(i, width=12)
                                     for cname in self.columns))
        return "\n".join(lines)

    def append_column(self, column: Column, name:Type.String="") -> None:
        if name == "":
          name = str(len(self._columns.keys()))
        elif name in self.columns:
          raise KeyError("Není povoleno modifikovat existujicí sloupce. Pro cílenou modifikaci použijte metodu mutate")
        self._columns[name]=column.copy()
        print(self._columns)

    def mutate(self, column: Column, name:str) -> 'DataFrame':
      # vytvořit nový dataframe (stejné vlastnosti jako aktuální) kopie
      # modifikace sloupce podobná jako v append_column
      # return df_copy
      df_copy=self.copy()
      if name in df_copy.columns:
        df_copy._columns[name]=column
      else:
        raise KeyError("Nelze modifikovat neexistující sloupec. Pro přidání sloupce použijte metodu append_column")
      return df_copy

    def copy(self) -> 'DataFrame':
      return DataFrame(self._columns)

    def append_row(self, row: Iterable) -> None:
      """nutno ošetřit situaci, kdy přidávaný řádek má jiný počet hodnot než je počet sloupců tabulky
        - možná řešení:
          -> 1) vyhodíme výjimku ValueError(není možné pridat špatný počet záznamů)
          -> 2) pokud je řádek kratší, doplníme hodnoty None nebo NA, pokud je řádek delší, usekneme hodnoty:
               -> uživatele upozorníme varovnou zprávou
          -> 3) pokud je řádek kratší, hodnoty cyklicky doplňujeme tak, aby nikde nebyly chybějící záznamy
              -> upozorníme uživatele
      """
      # zvolíme strategii 1)
      # nejprve kontrola počtu záznamů
      # iterace přes každý záznam iterable a přes všechny klíče dataframu
      # pro každý klíč zavoláme sloupci append a přidáme hodnotu řádku
      #for i in range(len(row)):
      #  self._columns[self.columns[i]].append(row[i])

      # kontrola špatného počtu hodnot
      if len(row) != len(self.columns):
        raise WrongSizeException("Zadaný řádek neodpovídá počtu sloupců")

      for value,key in zip(row, self.columns):
        self._columns[key].append(value)
      self._size+=1 # zvětšíme počet řádků v dataframu


    def filter(self, col_name:str, predicate: Callable[[Union[int, str]], bool]) -> 'DataFrame':
        pass

    def sort(self, col_name:str, ascending=True) -> 'DataFrame':
        # aplikujete řadící algoritmus na zvolený sloupec
        # při řazení je nutné pamatovat si indexy řádů každé z hodnot
        # výsledkem řazení je seznam indexů řádků, jak mají jít ve správném pořadí
        # vytvoříte nový datafram (kopii), na každý sloupec zavoláte permute(serazene_indexy)
        # vrátíte nový datafram

        indices = self.__merge_sort(self._columns[col_name]) # získáme indexy seřazených prvků
        df_new = self.copy() # vytvoříme kopii dataframu
        # pro každý sloupec dataframu zavolat permute
        for key in self.columns:
          df_new = df_new.mutate(df_new._columns[key].permute(indices),key)

        return df_new

    # @staticmethod
    # def __merge_sort(data: List[int]) -> List[int]:
    #     if len(data) <= 1:
    #         return data

    #     # Create a list of tuples where each tuple contains the index and the corresponding value
    #     indexed_data = [(index, value) for index, value in enumerate(data)]
        
    #     # Sort the indexed_data based on the values
    #     sorted_indexed_data = sorted(indexed_data, key=lambda x: x[1])

    #     # Extract the sorted indexes from the sorted_indexed_data
    #     sorted_indexes = [index for index, _ in sorted_indexed_data]

    #     return sorted_indexes

    @staticmethod
    def __merge_sort(data: List[int]) -> List[int]:
        if len(data) <= 1:
            return data

        # Vynechat None hodnoty
        index_data = [(index, value) for index, value in enumerate(data)]

        arr = [index for index, _ in index_data]
        print(arr)
        
        middle = len(arr) // 2
        left = DataFrame.__merge_sort(arr[:middle])
        right = DataFrame.__merge_sort(arr[middle:])
        print(left, right)
        return DataFrame.__merge(left, right)

    @staticmethod
    def __merge(left, right):
        result = []
        i = j = 0
        while i < len(left) and j < len(right):
            if left[i] < right[j]:
                result.append(left[i])
                i += 1
            else:
                result.append(right[j])
                j += 1
        result.extend(left[i:])
        result.extend(right[j:])
        return result

    def describe(self) -> str:
        """
        similar to pandas but only with min, max and avg statistics for floats and count"
        :return: string with decription
        """
        descriptions = []
        for col_name, col_data in self._columns.items():
            if col_data.dtype == Type.Float:
                valid_data = [value for value in col_data if value is not None]
                if valid_data:
                    descriptions.append("{}: Min={}, Max={}, Avg={}, Count={}".format(
                        col_name, min(valid_data), max(valid_data), sum(valid_data) / len(valid_data), len(valid_data)
                    ))
        
        return "\n".join(descriptions)

    def inner_join(self, other: 'DataFrame', self_key_column: str,
                   other_key_column: str) -> 'DataFrame':
        """
            Inner join between self and other dataframe with join predicate
            `self.key_column == other.key_column`.

            Possible collision of column identifiers is resolved by prefixing `_other` to
            columns from `other` data table.
        """
        joined_columns = {}

        for col_name, col_data in self._columns.items():
            if col_name == self_key_column:
                joined_columns[col_name] = col_data
            else:
                joined_columns["_other_" + col_name] = col_data

        for col_name, col_data in other._columns.items():
            if col_name == other_key_column:
                joined_columns[col_name] = col_data
            else:
                joined_columns["_other_" + col_name] = col_data

        return DataFrame(joined_columns)

    @staticmethod
    def read_csv(path: Union[str, Path]) -> 'DataFrame':
        return CSVReader(path).read()

    @staticmethod
    def read_json(path: Union[str, Path]) -> 'DataFrame':
        return JSONReader(path).read()


class Reader(ABC):
    def __init__(self, path: Union[Path, str]):
        self.path = Path(path)
    @abstractmethod
    def read(self) -> 'DataFrame':
        raise NotImplemented("Abstract method")


class JSONReader(Reader):
    def read(self) -> 'DataFrame':
        with open(self.path, "rt") as f:
            json = load(f)
        columns = {}
        for cname in json.keys():
            dtype = Type.Float if all(value is None or isinstance(value, Real)
                                      for value in json[cname]) else Type.String
            columns[cname] = Column(json[cname], dtype)
        return DataFrame(columns)


class CSVReader(Reader):
    def read(self) -> 'DataFrame':
        pass




class WrongSizeException(Exception):

  def __init__(self, zprava) -> None:
      super().__init__(zprava)



In [109]:
if __name__ == "__main__":
    df = DataFrame(dict(
        a=Column([None, 3.1415], Type.Float),
        b=Column(["a", 2], Type.String),
        c=Column(range(2), Type.Float)
        ))

    #df = DataFrame.read_json("data.json")

    d=Column(["Ota","Pavel"],dtype=Type.String)

    df.append_column(d)
    df.append_column(d)
    df.append_column(d,"jména")
    df.append_column(d)
    e=Column(["Franta","Pepa"],dtype=Type.String)


    radek = [1,2,3,4,5,6,7]
    df.append_row(radek)
    print(df)

    print(df.describe())

    for value in df._columns['jména']:
      print(type(value))

    # Test the sort method
    sorted_df = df.sort('c')  # Sort based on column 'a'
    print("\nSorted DataFrame:")
    print(sorted_df)


{'a': <__main__.Column object at 0x000002552510C810>, 'b': <__main__.Column object at 0x000002552510E950>, 'c': <__main__.Column object at 0x000002552510F750>, '3': <__main__.Column object at 0x000002552510D390>}
{'a': <__main__.Column object at 0x000002552510C810>, 'b': <__main__.Column object at 0x000002552510E950>, 'c': <__main__.Column object at 0x000002552510F750>, '3': <__main__.Column object at 0x000002552510D390>, '4': <__main__.Column object at 0x000002552510F2D0>}
{'a': <__main__.Column object at 0x000002552510C810>, 'b': <__main__.Column object at 0x000002552510E950>, 'c': <__main__.Column object at 0x000002552510F750>, '3': <__main__.Column object at 0x000002552510D390>, '4': <__main__.Column object at 0x000002552510F2D0>, 'jména': <__main__.Column object at 0x000002552510C8D0>}
{'a': <__main__.Column object at 0x000002552510C810>, 'b': <__main__.Column object at 0x000002552510E950>, 'c': <__main__.Column object at 0x000002552510F750>, '3': <__main__.Column object at 0x0000